In [2]:
import pandas as pd
import os

In [3]:
files_dir = "world_bank_projects"
files_list = os.listdir(files_dir)

In [8]:
tables = []
for f in files_list:
    if f[-4:] == 'xlsx':
        table = pd.read_excel(os.path.join(files_dir,f),skiprows=1)
        tables.append(table)

In [10]:
tables[0].head()

,Project Id,Region,Country,Project Status,Project Name,Project Development Objective,Implementing Agency,Consultant Services Required,Project URL,Board Approval Date,...,Grant Amount,Borrower,Lending Instrument,Environmental Assessment Category,Environmental and Social Risk,Sector 1,Sector 2,Sector 3,Theme 1,Theme 2
0,P174150,Latin America and Caribbean,United Mexican States,Active,Mexico Strengthening Economic Sustainability DPF,To facilitate economic resilience and support ...,MINISTRY OF FINANCE AND PUBLIC CREDIT (SECRETA...,NaN,https://projects.worldbank.org/en/projects-ope...,2021-01-21T00:00:00Z,...,0,UNITED MEXICAN STATES,Development Policy Lending,NaN,NaN,Sub-National Government,Insurance and Pension,Banking Institutions,NaN,NaN
1,P173367,Middle East and North Africa,Lebanese Republic,Active,LEBANON EMERGENCY CRISIS AND COVID-19 RESPONSE...,The objective of the Project is to: (a) provid...,Prime Minister's Office,NaN,https://projects.worldbank.org/en/projects-ope...,2021-01-12T00:00:00Z,...,0,Lebanese Republic,Investment Project Financing,NaN,Substantial,Social Protection,Public Administration - Social Protection,NaN,NaN,NaN
2,P173480,Latin America and Caribbean,Republic of Guatemala,Active,Responding to COVID-19: Modern and Resilient ...,The objective of the Project is to promote an ...,Ministry of Economy Guatemala (MINECO),No,https://projects.worldbank.org/en/projects-ope...,2021-01-08T00:00:00Z,...,0,Ministry of Finance Guatemala (MINFIN),Investment Project Financing,NaN,Substantial,"Agricultural markets, commercialization and ag...","Agricultural Extension, Research, and Other Su...","Public Administration - Industry, Trade and Se...",NaN,NaN
3,P174144,Europe and Central Asia,Republic of Turkey,Active,Turkey Rapid Support for Micro and Small Enter...,The Project Development Objective (PDO) is to ...,Small and Medium Enterprises Development Organ...,NaN,https://projects.worldbank.org/en/projects-ope...,2020-12-18T00:00:00Z,...,0,Small and Medium Enterprises Development Organ...,Investment Project Financing,NaN,Moderate,Manufacturing,Other Public Administration,NaN,NaN,NaN
4,P173698,Latin America and Caribbean,Republic of Guatemala,Active,Crisis Response and Recovery in Guatemala DPL,This DPF series supports the Government’s effo...,Ministry of Finance Guatemala (MINFIN),NaN,https://projects.worldbank.org/en/projects-ope...,2020-12-17T00:00:00Z,...,0,Republic of Guatemala,Development Policy Lending,NaN,NaN,Central Government (Central Agencies),Primary Education,Trade,NaN,NaN


In [16]:
table = pd.concat(tables,ignore_index=True)

In [22]:
table.columns

Index(['Project Id', 'Region', 'Country', 'Project Status', 'Project Name',
       'Project Development Objective', 'Implementing Agency',
       'Consultant Services Required', 'Project URL', 'Board Approval Date',
       'Project Closing Date', 'Financing Type', 'Current Project Cost',
       'IBRD Commitment', 'IDA Commitment', 'Total IDA and IBRD Commitment',
       'Grant Amount', 'Borrower', 'Lending Instrument',
       'Environmental Assessment Category', 'Environmental and Social Risk',
       'Sector 1', 'Sector 2', 'Sector 3', 'Theme 1', 'Theme 2'],
      dtype='object')

In [27]:
table['IBRD Commitment'] = table['IBRD Commitment'].apply(lambda x: x.replace(',',''))

In [29]:
table['IBRD Commitment'] = table['IBRD Commitment'].astype(float)

In [30]:
table.describe()

,Financing Type,Current Project Cost,IBRD Commitment,Grant Amount
count,0.0,0.0,4.000000e+03,4000.0
mean,NaN,NaN,1.953895e+08,0.0
std,NaN,NaN,1.961879e+08,0.0
min,NaN,NaN,0.000000e+00,0.0
25%,NaN,NaN,5.013000e+07,0.0
50%,NaN,NaN,1.300000e+08,0.0
75%,NaN,NaN,2.735800e+08,0.0
max,NaN,NaN,1.450000e+09,0.0


In [33]:
table['IDA Commitment'] = table['IDA Commitment'].apply(lambda x: x.replace(',',''))

In [34]:
table['IDA Commitment'] = table['IDA Commitment'].astype(float)

In [35]:
table.describe()

,Financing Type,Current Project Cost,IBRD Commitment,IDA Commitment,Grant Amount
count,0.0,0.0,4.000000e+03,4.000000e+03,4000.0
mean,NaN,NaN,1.953895e+08,7.301348e+06,0.0
std,NaN,NaN,1.961879e+08,5.150424e+07,0.0
min,NaN,NaN,0.000000e+00,0.000000e+00,0.0
25%,NaN,NaN,5.013000e+07,0.000000e+00,0.0
50%,NaN,NaN,1.300000e+08,0.000000e+00,0.0
75%,NaN,NaN,2.735800e+08,0.000000e+00,0.0
max,NaN,NaN,1.450000e+09,7.500000e+08,0.0


In [38]:
table[['Country','Project Name','Board Approval Date','Borrower','IBRD Commitment','Sector 1']].sample(10)

,Country,Project Name,Board Approval Date,Borrower,IBRD Commitment,Sector 1
1397,Republic of Serbia,State Owned Financial Institutions Strengtheni...,2018-03-20T00:00:00Z,Republic of Serbia,50000000.0,Public Administration - Financial Sector
3267,Republic of Colombia,"Second Fiscal Sustainability, Competitiveness,...",2019-05-21T00:00:00Z,NaN,718500000.0,Central Government (Central Agencies)
2163,Republic of India,National Program for Improving the Quality of ...,2020-03-11T00:00:00Z,Republic of India,30000000.0,Central Government (Central Agencies)
1775,Romania,Disaster Risk Management Development Policy Lo...,2018-06-26T00:00:00Z,Romania (through its Ministry of Public Finance),493060000.0,Other Public Administration
1416,Montenegro,Montenegro First Fiscal and Financial Sector R...,2017-12-20T00:00:00Z,Ministry of Finance,93000000.0,Central Government (Central Agencies)
199,Kingdom of Eswatini,Network Reinforcement and Access Project,2019-06-27T00:00:00Z,Kingdom of Eswatini,40000000.0,Energy Transmission and Distribution
172,Republic of Serbia,Real Estate Management Project Additional Fina...,2019-11-21T00:00:00Z,Republic of Serbia,22900000.0,Other Public Administration
2164,Bosnia and Herzegovina,Real Estate Registration Project - Additional ...,2020-03-05T00:00:00Z,Bosnia and Herzegovina,22400000.0,Other Public Administration
3756,Eastern Africa,Agricultural Productivity Program for Southern...,2018-12-18T00:00:00Z,"Leostho Ministry of Finance, Angola Ministry o...",25000000.0,"Agricultural Extension, Research, and Other Su..."
1722,Republic of India,Innovation in Solar Power and Hybrid Technologies,2019-03-29T00:00:00Z,DEPARTMENT OF ECONOMIC AFFAIRS,150000000.0,Renewable Energy Solar


In [39]:
table['Sector 1'].describe()

count                                      3924
unique                                       59
top       Central Government (Central Agencies)
freq                                        365
Name: Sector 1, dtype: object